<a href="https://colab.research.google.com/github/vitaliy-sharandin/data_science_projects/blob/master/portfolio/nlp/fine-tuned-llm/wiseai.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Psy AI
Psy AI is a Llama3-8b model instruction fine-tuned on depression dataset and is meant to help improve mental well-being.

In future iterations it is meant to be trained on multiple philosophical and psychological datasets in order to provide multifaceted answers to complex mental health issues.



# Datasets

Philosophy datasets (* for future training)
* https://www.kaggle.com/datasets/christopherlemke/philosophical-texts
* https://www.workwithdata.com/object/philosophy-science-complete-a-text-on-traditional-problems-schools-thought-book-by-edwin-h-c-hung-0000
* https://www.kaggle.com/datasets/christopherlemke/philosophy-authors-writings-german
* https://www.workwithdata.com/object/philosophical-inquiries-an-introduction-to-problems-philosophy-book-by-nicholas-rescher-0000
* https://www.workwithdata.com/object/roman-stoicism-book-by-edward-vernon-arnold-1857
* https://www.workwithdata.com/object/wisdom-energy-basic-buddhist-teachings-book-by-thubten-yeshe-1935

Psychology and mental health datasets

* Kaggle Depression data for chatbot https://www.kaggle.com/datasets/nupurgopali/depression-data-for-chatbot
* Kaggle Psychometrics dataset https://www.kaggle.com/discussions/general/304994
* Psychometric tests dataset https://ieee-dataport.org/documents/psychometric-tests-dataset
* Psychometric NLP https://paperswithcode.com/dataset/psychometric-nlp
* Reddit mental health dataset https://zenodo.org/record/3941387
* Reddit mental disorders identification https://www.kaggle.com/datasets/kamaruladha/mental-disorders-identification-reddit-nlp
* Kaggle Mental Health Conversational Data https://www.kaggle.com/datasets/elvis23/mental-health-conversational-data
* Kaggle Mental Health FAQ for Chatbot https://www.kaggle.com/narendrageek/mental-health-faq-for-chatbot/code
* A human consciousness questionnaire dataset https://data.mendeley.com/datasets/69p62ksdh6
* paperswithcode Self-reported Mental Health Diagnoses https://paperswithcode.com/dataset/smhd
* paperswithcode Mental Health Summarization Dataset https://paperswithcode.com/dataset/mentsum
* HuggingFace psychology dataset https://huggingface.co/datasets/samhog/psychology-10k


In [0]:
!pip install -U -q gradio
!pip install -U -q datasets
!pip install -U -q bitsandbytes
!pip install -U -q git+https://github.com/huggingface/transformers
!pip install -q -U peft
!pip install -q -U accelerate
!pip install -U -q trl
!pip install -U -q torch torchvision
!pip install -U -q accelerate
!pip install -U -q flash-attn

!pip install -U -q evaluate
!pip install -U -q rouge_score
!pip install -U -q optuna

!pip install -U -q urllib3==1.26.7
!pip install -U -q mlflow-skinny==2.12.1

In [0]:
dbutils.library.restartPython()

In [0]:
from datetime import datetime
from datasets import load_dataset
import json
import yaml
import gradio as gr
import torch
import transformers
from transformers import GenerationConfig, Trainer, TrainingArguments, AutoTokenizer, AutoModelForCausalLM, BitsAndBytesConfig, pipeline
from trl import SFTTrainer, DataCollatorForCompletionOnlyLM
from datasets import Dataset
from peft import LoraConfig, prepare_model_for_kbit_training, get_peft_model, PeftModel, AutoPeftModelForCausalLM
import numpy as np
from evaluate import load
import optuna
import mlflow

import warnings
warnings.filterwarnings('ignore')

In [0]:
catalog = "model_registry"
schema = "default"
model_name = "psy-ai"
mlflow.set_registry_uri("databricks-uc")
mlflow.register_model(
    model_uri="runs:/bcae496e43a0496da7a6eafe0ab569d8/NousResearch/Meta-Llama-3-8B-Instruct-peft-trained",
    name=f"{catalog}.{schema}.{model_name}"
)

# Dataset instruction transformation

Depression dataset with 51 q&a entries was taken for mvp to see if model is indeed training.

In [0]:
depression_dataset = load_dataset("vitaliy-sharandin/depression-instruct")

First, we modify our dataset to Alpaca format and create two datasets. One - for testing and evaluation and second one for inference, where responses are not available in formatted prompt.

In [0]:
def formatting_func_train(example):
  if example.get("context", "") != "":
      input_prompt = (f"Below is an instruction that describes a task, paired with an input that provides further context. "
      "Write a response that appropriately completes the request.\n\n"
      "### Instruction:\n"
      f"{example['instruction']}\n\n"
      f"### Input: \n"
      f"{example['context']}\n\n"
      f"### Response: \n"
      f"{example['response']}")

  else:
    input_prompt = (f"Below is an instruction that describes a task. "
      "Write a response that appropriately completes the request.\n\n"
      "### Instruction:\n"
      f"{example['instruction']}\n\n"
      f"### Response:\n"
      f"{example['response']}")

  return {"text" : input_prompt}

def formatting_func_test(example):
  if example.get("context", "") != "":
      input_prompt = (f"Below is an instruction that describes a task, paired with an input that provides further context. "
      "Write a response that appropriately completes the request.\n\n"
      "### Instruction:\n"
      f"{example['instruction']}\n\n"
      f"### Input: \n"
      f"{example['context']}\n\n"
      f"### Response: \n")

  else:
    input_prompt = (f"Below is an instruction that describes a task. "
      "Write a response that appropriately completes the request.\n\n"
      "### Instruction:\n"
      f"{example['instruction']}\n\n"
      f"### Response: \n")

  return {"text" : input_prompt}

In [0]:
formatted_depression_dataset_train = depression_dataset.map(formatting_func_train)
formatted_depression_dataset_test = depression_dataset.map(formatting_func_test)

# Model load

We are loading our Llama2 model in 4bit quantized form as well as applying Lora for peft training.

In [0]:
model_name = "NousResearch/Meta-Llama-3-8B-Instruct"

def get_tokenizer():
  tokenizer = AutoTokenizer.from_pretrained(model_name)
  tokenizer.pad_token = tokenizer.eos_token
  tokenizer.padding_side = "right"
  return tokenizer

def get_model():

  bnb_config = BitsAndBytesConfig(
      load_in_4bit=True,
      bnb_4bit_use_double_quant=True,
      bnb_4bit_quant_type="nf4",
      bnb_4bit_compute_dtype=torch.bfloat16
  )

  qlora_config = LoraConfig(lora_alpha=16,
                          lora_dropout=0.1,
                          r=64,
                          bias="none",
                          task_type="CAUSAL_LM")

  base_training_model = AutoModelForCausalLM.from_pretrained(
      model_name,
      quantization_config=bnb_config,
      device_map = {"": 0},
      offload_folder = f"output/{model_name}"
  )
  
  base_training_model.gradient_checkpointing_enable()
  base_training_model = prepare_model_for_kbit_training(base_training_model)
  base_training_model = get_peft_model(base_training_model, qlora_config)
  return base_training_model.to('cuda')

base_training_model = get_model()
tokenizer = get_tokenizer()

torch.manual_seed(42)
print(base_training_model)

# Model instruction fine-tuning

Here we are defining inference function which returns bleu, rouge and f1 metrics after comparison of predicted and reference responses. My tests have shown that standard trainer `compute_metrics` method is quite inefficient and is not quite suitable for instruction fine-tuning during manual observations or generated results.

In [0]:
model_trained_checkpoint = f"{model_name}-peft-trained"
catalog = "model-registry"
schema = "default"
model_name = "psy-ai"

def run_inference(model, test_dataset, test_size=None):

  if test_size:
    test_dataset = test_dataset['train'].select(range(test_size))
  else:
    test_dataset = test_dataset['train']

  inputs = test_dataset['text']
  reference_responses = test_dataset['response']

  decoded_predictions = []
  for prompt in inputs:
    pipe = pipeline("text-generation", model=model, tokenizer=tokenizer)
    prediction = pipe(prompt, max_length=200, top_p=0.9, temperature=0.9, num_return_sequences=1, return_full_text=False)[0]['generated_text']
    decoded_predictions.append(prediction[:str(prediction).find("###")])

  for input, pred, label in zip(inputs[:3], decoded_predictions[:3], reference_responses[:3]):
    print("[Input]:\n\n", input)
    print("[Prediction]:\n\n", pred)
    print("[Reference response]:\n\n", label)
    print("----\n\n")

  bleu = load("bleu")
  bleu_results = bleu.compute(predictions=decoded_predictions, references=reference_responses)

  rouge = load('rouge')
  rouge_results = rouge.compute(predictions=decoded_predictions, references=reference_responses)

  f1 = 2 * (bleu_results['bleu'] * rouge_results['rouge1']) / (bleu_results['bleu'] + rouge_results['rouge1'])

  scores = {
        "bleu": bleu_results["bleu"],
        "rouge1": rouge_results["rouge1"],
        "rouge2": rouge_results["rouge2"],
        "rougeL": rouge_results["rougeL"],
        "f1": f1
    }

  return scores

def bleu_rouge_f1_metrics(eval_pred):
  predictions, labels = eval_pred
  predictions = np.argmax(predictions, axis=-1)

  labels = [[idx for idx in label if idx != -100] for label in labels]
  predictions = [[idx for idx in prediction if idx != -100] for prediction in predictions]

  decoded_predictions = tokenizer.batch_decode(predictions, skip_special_tokens=True)
  decoded_labels = tokenizer.batch_decode(labels, skip_special_tokens=True)

  bleu = load("bleu")
  bleu_results = bleu.compute(predictions=decoded_predictions, references=decoded_labels)

  rouge = load('rouge')
  rouge_results = rouge.compute(predictions=decoded_predictions, references=decoded_labels)

  f1 = 2 * (bleu_results['bleu'] * rouge_results['rouge1']) / (bleu_results['bleu'] + rouge_results['rouge1'])

  scores = {
        "bleu": bleu_results["bleu"],
        "rouge1": rouge_results["rouge1"],
        "rouge2": rouge_results["rouge2"],
        "rougeL": rouge_results["rougeL"],
        "f1": f1
    }

  return scores

def fine_tune(model, tokenizer, train_dataset, eval_dataset, metrics_func, only_evaluate=False):
  model.gradient_checkpointing_enable()
  supervised_finetuning_trainer = SFTTrainer(model=model,
                                            train_dataset=train_dataset,
                                            eval_dataset=eval_dataset,
                                            args=TrainingArguments(
                                                report_to="mlflow",
                                                run_name=f"{model_name}-{datetime.now().strftime('%Y-%m-%d-%H-%M-%s')}",
                                                output_dir="output/training_results",
                                                per_device_train_batch_size=8,
                                                per_device_eval_batch_size=1,
                                                gradient_accumulation_steps=2,
                                                ddp_find_unused_parameters=False,
                                                optim="paged_adamw_8bit",
                                                save_steps=1000,
                                                logging_steps=30,
                                                learning_rate=2e-4,
                                                weight_decay=0.001,
                                                fp16=False,
                                                bf16=False,
                                                max_grad_norm=0.3,
                                                max_steps=-1,
                                                warmup_ratio=0.3,
                                                group_by_length=True,
                                                lr_scheduler_type="constant"
                                            ),
                                            dataset_text_field="text",
                                            max_seq_length=2048,
                                            compute_metrics=metrics_func,
                                            data_collator=transformers.DataCollatorForLanguageModeling(tokenizer, mlm=False))

  if only_evaluate:
    mlflow.autolog(disable=True)
    return supervised_finetuning_trainer.evaluate()

  else:
    mlflow.autolog(disable=False)
    
    mlflow.set_experiment('/Shared/psy-ai')
    with mlflow.start_run() as run:

      supervised_finetuning_trainer.train()

      model_info = mlflow.transformers.log_model(
                    transformers_model={"model": supervised_finetuning_trainer.model, "tokenizer": tokenizer},
                    task="text-generation",
                    save_pretrained=True,
                    artifact_path=model_trained_checkpoint)
      
      mlflow.set_registry_uri("databricks-uc")
      mlflow.register_model(
          model_uri=model_info.model_uri,
          name=f"{catalog}.{schema}.{model_name}"
      )

## Evaluating base model

In [0]:
# params = {
#     "model": base_training_model,
#     "tokenizer": tokenizer,
#     "train_dataset": formatted_depression_dataset_train["train"],
#     "eval_dataset": formatted_depression_dataset_train["train"],
#     "metrics_func": bleu_rouge_f1_metrics,
#     "only_evaluate": True
# }

# fine_tune(**params)

Evaluation metrics are quite low before training.

Let's train the model and see how metrics and responses change.

## Fine-tuning base model and registering it to Unity Catalogue model registry


In [0]:
params = {
    "model": base_training_model,
    "tokenizer": tokenizer,
    "train_dataset": formatted_depression_dataset_train["train"],
    "eval_dataset": formatted_depression_dataset_train["train"],
    "metrics_func": bleu_rouge_f1_metrics
}

fine_tune(**params)

# Deploying model to Databricks Serving Endpoints


# Gradio demo in Hugging Face Spaces

https://huggingface.co/spaces/vitaliy-sharandin/WiseAI

## Local launch
Requirements:
* Choose free T4 GPU in Colab environment settings
* Run `pip install` and `imports` cells at the beginning before launching Gradio.

In [0]:
model = AutoModelForCausalLM.from_pretrained(
      'vitaliy-sharandin/wiseai',
      load_in_8bit=True,
      device_map = {"": 0}
  )
tokenizer = AutoTokenizer.from_pretrained('vitaliy-sharandin/wiseai')

pipe = pipeline('text-generation', model=model,tokenizer=tokenizer)

def generate_text(instruction, input):
  if not instruction.strip():
    return str('The instruction field is required.')

  if instruction.strip() and input.strip():
    input_prompt = (f"Below is an instruction that describes a task. "
      "Write a response that appropriately completes the request.\n\n"
      "### Instruction:\n"
      f"{instruction}\n\n"
      "### Input:\n"
      f"{input}\n\n"
      f"### Response: \n")
  else :
    input_prompt = (f"Below is an instruction that describes a task. "
        "Write a response that appropriately completes the request.\n\n"
        "### Instruction:\n"
        f"{instruction}\n\n"
        f"### Response: \n")
  result = pipe(input_prompt, max_length=200, top_p=0.9, temperature=0.9, num_return_sequences=1, return_full_text=False)[0]['generated_text']
  return result[:str(result).find("###")]

iface = gr.Interface(fn=generate_text, inputs=[gr.Textbox(label="Instruction"),
                                               gr.Textbox(label="Additional Input")],
                     outputs=gr.Textbox(label="Response"))
iface.launch()